In [2]:
! pip install beautifulsoup4

In [4]:
! pip install request

  Stored in directory: /home/dsxuser/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /home/dsxuser/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /home/dsxuser/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /home/dsxuser/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /home/dsxuser/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request get post query-string public


In [244]:
from bs4 import BeautifulSoup
import requests

In [5]:
#request from the website
source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=920980179').text

In [6]:
#parse the html file
soup = BeautifulSoup(source)

In [246]:
#take a look at the html file
soup.table.tr.text

'\nPostcode\nBorough\nNeighbourhood\n'

In [57]:
with open('neighbors.csv', 'w') as f:
    for line in soup.table.find_all('tr'):
        line = line.text.replace('\n', ',')
        line = line[1:-1] + '\n'
        #print(line)
        f.write(line)

In [280]:
import pandas as pd
df = pd.read_csv('neighbors.csv')
df.shape

(288, 3)

In [281]:
df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [282]:
df['Neighbourhood'].value_counts()[:5]

Not assigned       78
St. James Town      2
Runnymede           2
Dufferin            1
L'Amoreaux East     1
Name: Neighbourhood, dtype: int64

In [116]:
import numpy as np

In [283]:
#replace 'Not assigned' value as NaN for easier processing
df.replace('Not assigned', np.nan, inplace = True)

In [284]:
#drop rows where Borough is NA
df.dropna(subset = ['Borough'], inplace = True)

In [285]:
#only one row has no Neighbourhood
df[df['Neighbourhood'].isna()]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,NaN


In [286]:
#mannually fill in its Borough
df.fillna('Queen\'s Park', inplace = True)

In [287]:
print(df.shape)
df.head()

(211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [288]:
df = df.add(['','',',']) #for easier str processing

In [289]:
df = df.groupby(['Postcode', 'Borough']).sum()

In [290]:
df['Neighbourhood'].values[:5]

array(['Rouge,Malvern,', 'Highland Creek,Rouge Hill,Port Union,',
       'Guildwood,Morningside,West Hill,', 'Woburn,', 'Cedarbrae,'],
      dtype=object)

In [291]:
df['Neighbourhood'].values[0][0:-1]

'Rouge,Malvern'

In [292]:
#remove the last comma and add a space after the first comma
new_col = []
for row in df['Neighbourhood'].values:
    row = row[0:-1].replace(',', ', ')
    new_col.append(row)

In [293]:
new_col[:5]

['Rouge, Malvern',
 'Highland Creek, Rouge Hill, Port Union',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae']

In [294]:
df['Neighbourhood'] = new_col

In [298]:
df.reset_index(inplace = True)

In [305]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
